In [ ]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb import OutputHandler
from tqdm import tqdm

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
cache_config = ConfigCache(enabled=False, density=False)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)
calc.opts.cache = cache_config
calc.opts.grad = False
OutputHandler.verbosity = 0


# WORKING!! Repeated setup from dxtb test

In [1]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from tqdm import tqdm
from tad_mctc.data.molecules import mols as samples
from dxtb import Calculator
from dxtb.config import ConfigCache

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}
cache_config = ConfigCache(enabled=False, density=True, fock=True)
opts = {"verbosity": 2}

# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
opts = dict(opts, **{"scf_mode": "implicit"})
dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config
calc.get_energy(pos, charges)
dxtb.timer.print(v=1)

print(f"Calc scf mode: {calc.opts.scf.scf_mode}")

SCF Energy  : -326.18650453723933 Hartree.
Total Energy: -322.95377994015860 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Calculator                  0.033           3.16
Classicals                  0.027           2.63
 - Repulsion                0.005          17.41
 - Halogen                  0.000           0.61
 - DispersionD3             0.022          81.37
Integrals                   0.117          11.19
 - Overlap                  0.085          72.94
 - Core Hamiltonian         0.032          27.00
SCF                         0.860          82.36
 - Interaction Cache        0.009           1.07
 - Potential                0.730          84.82
 - Fock build               0.014           1.62
 - Diagonalize              0.582          67.69
 - Cholesky                 0.022           2.57
 - inverse + T              0.082           9.57
 - A2 matmul                0.087          10.10
 - _degen_sym

Calc scf mode: 1


In [ ]:
for _ in tqdm(range(1000)):
    calc.get_energy(pos, charges)
    F = calc.cache["fock"]
    # torch.cuda.empty_cache()

In [ ]:
from dxtb import labels
from dxtb._src import scf
from dxtb.config import Config
from dxtb.integrals import DriverManager
from dxtb._src.integral.container import IntegralMatrices
from dxtb import IndexHelper
from dxtb._src.components.interactions.coulomb.secondorder import new_es2
from dxtb._src.components.interactions.coulomb.thirdorder import new_es3
from dxtb._src.components.interactions.list import InteractionList
from dxtb._src.xtb.gfn1 import GFN1Hamiltonian
from dxtb._src.constants import defaults

from dxtb.integrals import DriverManager
from dxtb.integrals.factories import new_overlap
from tad_mctc.convert import any_to_tensor

def trimmed_singlepoint(numbers, positions, result="density", chrg=defaults.CHRG, spin=defaults.SPIN):
    # Device and dtype setup
    device = positions.device
    dtype = positions.dtype
    dd = {'device': device, 'dtype': dtype}

    # Move charge and spin to the target device
    chrg = any_to_tensor(chrg, **dd).to(device)
    if spin is not None:
        spin = any_to_tensor(spin, **dd).to(device)

    # Config setup on the correct device
    opts = Config(**dd)
    opts.ints.level = labels.INTLEVEL_HCORE
    opts.exclude = set()  # Ensure 'scf' is not excluded

    par = dxtb.GFN1_XTB
    ihelp = IndexHelper.from_numbers(numbers, par).to(device)  # Move to device

    # Initialize integral matrices on device
    intmats = IntegralMatrices(**dd)

    # Overlap integral
    driver_name = 0  # libcint
    drv_mgr = DriverManager(driver_name, **dd)
    drv_mgr.create_driver(numbers.to(device), par, ihelp)  # Ensure numbers are on device
    drv_mgr.driver.setup(positions.to(device))

    ovlp_integral = new_overlap(drv_mgr.driver_type, **dd)
    ovlp_integral.build(drv_mgr.driver)

    # Ensure the overlap matrix is on the specified device
    intmats.overlap = ovlp_integral.matrix.to(device)  # Move overlap matrix to device

    # Hcore integral
    h0 = GFN1Hamiltonian(numbers.to(device), par, ihelp, **dd)  # Ensure numbers and ihelp on device
    hcore = h0.build(positions.to(device), intmats.overlap)  # Use the overlap matrix on the device
    intmats.hcore = hcore


    # Create interactions (es2 and es3) on the correct device
    es2 = (
        new_es2(numbers.to(device), par, **dd)
        if not {"all", "es2"} & opts.exclude
        else None
    )
    es3 = (
        new_es3(numbers.to(device), par, **dd)
        if not {"all", "es3"} & opts.exclude
        else None
    )
    interactions = InteractionList(es2, es3, **dd)

    # Build interaction cache on the correct device
    icaches = interactions.get_cache(numbers=numbers.to(device), positions=positions.to(device), ihelp=ihelp)

    scf_opts = opts.scf

    # Run SCF on the specified device
    scf_results = scf.solve(
        numbers.to(device),
        positions.to(device),
        chrg,
        spin,
        interactions,
        icaches,
        ihelp,
        scf_opts,
        intmats,
        h0.refocc.to(device) if h0.refocc is not None else None,
    )

    return scf_results["energy"].to(device)  # Return density on the specified device

# Run the calculation
scf_results = trimmed_singlepoint(numbers, pos, chrg=0, spin=None)



In [ ]:
from memory_profiler import profile

@profile
def run_energy():
    for _ in range(100):
        calc.get_energy(pos)

run_energy()

# Torch profiler

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
    with record_function("model_inference"):
        for _ in range(100):
            calc.get_energy(pos)



In [ ]:
print(print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10)))

# Trimmed down function

In [ ]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        for _ in tqdm(range(100)):
            trimmed_singlepoint(numbers, positions, result="density", chrg=0, spin=None)

In [ ]:
print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10))

In [ ]:
def trimmed_singlepoint_print(numbers, positions, chrg=defaults.CHRG, spin=defaults.SPIN):
    # Device and dtype setup
    device = positions.device
    dtype = positions.dtype
    dd = {'device': device, 'dtype': dtype}

    def print_memory(message):
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # Convert to MB
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # Convert to MB
        print(f"{message}: Allocated = {allocated:.2f} MB, Reserved = {reserved:.2f} MB")

    print_memory("Memory after setup")

    # Move charge and spin to the target device
    chrg = any_to_tensor(chrg, **dd).to(device)
    if spin is not None:
        spin = any_to_tensor(spin, **dd).to(device)
    print_memory("Memory after charge and spin")

    # Config setup on the correct device
    opts = Config(**dd)
    opts.ints.level = labels.INTLEVEL_HCORE
    opts.exclude = set()  # Ensure 'scf' is not excluded

    par = dxtb.GFN1_XTB
    ihelp = IndexHelper.from_numbers(numbers, par).to(device)  # Move to device
    print_memory("Memory after IndexHelper")

    # Initialize integral matrices on device
    intmats = IntegralMatrices(**dd)
    print_memory("Memory after IntegralMatrices")

    # Overlap integral
    driver_name = 0  # libcint
    drv_mgr = DriverManager(driver_name, **dd)
    drv_mgr.create_driver(numbers.to(device), par, ihelp)  # Ensure numbers are on device
    drv_mgr.driver.setup(positions.to(device))
    print_memory("Memory after DriverManager setup")

    ovlp_integral = new_overlap(drv_mgr.driver_type, **dd)
    ovlp_integral.build(drv_mgr.driver)
    intmats.overlap = ovlp_integral.matrix.to(device)  # Move overlap matrix to device
    print_memory("Memory after overlap integral")

    # Hcore integral
    h0 = GFN1Hamiltonian(numbers.to(device), par, ihelp, **dd)
    hcore = h0.build(positions.to(device), intmats.overlap)
    intmats.hcore = hcore
    print_memory("Memory after Hcore integral")

    # Create interactions (es2 and es3) on the correct device
    es2 = (
        new_es2(numbers.to(device), par, **dd)
        if not {"all", "es2"} & opts.exclude
        else None
    )
    es3 = (
        new_es3(numbers.to(device), par, **dd)
        if not {"all", "es3"} & opts.exclude
        else None
    )
    interactions = InteractionList(es2, es3, **dd)
    print_memory("Memory after interaction list")

    # Build interaction cache on the correct device
    icaches = interactions.get_cache(numbers=numbers.to(device), positions=positions.to(device), ihelp=ihelp)
    print_memory("Memory after interaction cache")

    scf_opts = opts.scf

    # Run SCF on the specified device
    scf_results = scf.solve(
        numbers.to(device),
        positions.to(device),
        chrg,
        spin,
        interactions,
        icaches,
        ihelp,
        scf_opts,
        intmats,
        h0.refocc.to(device) if h0.refocc is not None else None,
    )
    print_memory("Memory after SCF")
    print()

    return scf_results["density"].to(device)  # Return density on the specified device



In [ ]:
for _ in range(10):
    trimmed_singlepoint_print(numbers, pos, chrg=0, spin=None)